# Lesson2. Relational Query Processing

----

## Topic1. Query Plan Generation

### 1-1. Query Plan

사람이 쉽게 인식할 수 있도록 쿼리를 작성하지만 컴퓨터는 다른 방식으로 인식한다.

1. order 테이블의 order와 item 테이블의 order를 스캔한 후
1. order의 내용으로 두 테이블을 결합
1. 일반적인 C 코드로 작성하는 것보다 훨씬 빠름

#### SQL 코드
``` mysql
SELECT *
FROM Order o, Item i
WHERE o.order = i.order
```

#### C코드 1
이중 반복문으로 처리하여 작성
- 테이블의 크기만큼의 시간이 소요

``` c
for each record i in Item:
    for each record o in Order:
        if o.order = i.order:
            return (r,s)
```

#### C코드 2
해쉬 코드를 사용할 수 도 있다.
- item 테이블 + Order 테이블 만큼의 시간이 소요

``` c
for each record i in Item:
    insert into hashtable
    
for each record o in Order:
    lookup corresponding records in hashtable
    return matching pairs
```

### 1-2. Exposing the Algebra: Micosoft SQL Server

사용자는 어떤 쿼리가 최적인지 알 필요가 없다. 그 계산은 DBMS가 계산하여 최적의 쿼리를 정하여 계산한다.

#### SQL 1

- **셀프조인** : 자기 자신을 조인하는 것, 아래 예에서는 term_id가 매칭이 안된는 것만 조회
- 해쉬조인을 할지 그냥 조인을 할지는 상황에 맞추어서 실행됨

```mysql
SELECT a.term_id, b.term_id
FROM [billhowe].[reuters] a, [billhowe].[reuters] b
WHERE a.doc_id = b.doc_id 
    AND a.term_id != b.term_id
```

#### SQL2

- 위의 쿼리식에 a.term_id 조건이 하나 추가됨
- 위 조건으로 실행 조인이 달라짐
- DBMS에서 해쉬보다는 네스트 조인이 더 빠르다고 판단

```mysql
SELECT a.term_id, b.term_id
FROM [billhowe].[reuters] a, [billhowe].[reuters] b
WHERE a.doc_id = b.doc_id 
    AND a.term_id != b.term_id
    AND a.term_id = 'parliament'
```

### 1-3. Algebra Optimization

로직에 따라 같은 결과라도 처리 비용이 달라질 수 있다.
- 괄호() 기호에 따라 처리 순서가 달라진다.
- **Declarative 언어** : 어떻게 할지만을 정하고 어떤 순서로 실행할지는 정하지 않는 언어.


#### SQL 1
SQL을 보면 어떤게 먼저 처리될지 알 수 없다.
- 쿼리 플랜을 보면 테이블을 먼저 스캔한 후, SELECT로 요소를 선택 후 JOIN한다.
``` mysql
SELECT *
FROM Order o, Item i
WHERE o.order = i.order
AND o.date = today()
```

#### SQL2
SQL은 무엇이지 어떻게가 아니다. SQL is the "WHAT" not the "HOW"
- 무엇을 얻으려는지 알 수 있다. 하지만 어떤 순서인지는 알 수 없다.

``` mysql
SELECT DISTINCT x.name, z.name
FROM Product x, Purchase y, Customer z
WHERE x.pid = y.pid
AND y.cid = z.cid
AND x.price > 100
AND z.city = 'Seoul'
```

----

## Topic 2. View And Index

### 2-1. View

테이블을 통해 또 다른 가상의 테이블을 만들수 있다. 
- 쿼리에서 처리한 결과 또한 테이블 형식으로 나옴
- 그 결과를 사용하기 위해 View가 필요

### 2-2. Example : View
View를 사용하여 가상의 테이블을 만들 수 있다.

``` mysql
CREATE VIEW StorePrice AS
SELECT x.store, y.price
FROM Purchase x, Product y
WHERE x.pid = y.pid
```

- 위 쿼리식의 결과로 StorePrice라는 VIew(가상의 테이블)가 생성

### 2-3. How to User a View?

복잡한 테이블이나 조건을 보다 단순화 하기 위해 사용한다.

``` mysql
SELECT DISTINCT z.name, u.sotre AS
FROM Coustomer z, Purchase u, StorePrice v
WHERE z.cid = u.cid
AND u.store = v.store
AND v.price > 1000

```

- StorePrice라는 가상의 View를 사용하여 복잡한 식을 줄임

### 2-4. Index

DBMS는 필요한 정보를 빠르게 검색할 수 있다. 인덱스를 만들면 나중에 더 빠르고 효율적으로 필요한 자료를 찾을 수 있다.

``` mysql
CREATE INDEX seq_idx ON sequence (seq);

SELECT seq
FROM sequence
WHERE seq = 'GATTACGATATTA';
```


----